# Anomalia de campo magnético anômalo

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html) para calcular a amplitude do campo magnético (Sarah e Li, 2004; Li et al., 2010; Li e Li, 2014; Leão-Santos et al., 2015) produzida por prismas com seção horizontal poligonal.

* Sarah E. Shearer and Yaoguo Li (2004) 3D Inversion of magnetic total 
    gradient data in the presence of remanent magnetization. SEG Technical
    Program Expanded Abstracts 2004: pp. 774-777.
    doi: [10.1190/1.1851318](http://dx.doi.org/10.1190/1.1851318)

* Yaoguo Li, Sarah E. Shearer, Matthew M. Haney, and Neal Dannemiller (2010). 
    Comprehensive approaches to 3D inversion of magnetic data affected by remanent 
    magnetization. 
    Geophysics, 75(1), L1-L11.
    doi: [10.1190/1.3294766](http://dx.doi.org/10.1190/1.3294766)
    
* Shu-Ling Li and Yaoguo Li (2014). Inversion of magnetic anomaly on rugged 
    observation surface in the presence of strong remanent magnetization. 
    Geophysics, 79(2), J11-J19. 
    doi: [10.1190/geo2013-0126.1](http://dx.doi.org/10.1190/geo2013-0126.1)

* Marcelo Leão-Santos, Yaoguo Li, and Roberto Moraes (2015). 
    Application of 3D magnetic amplitude inversion to iron oxide-copper-gold 
    deposits at low magnetic latitudes: A case study from Carajás Mineral 
    Province, Brazil. 
    Geophysics, 80(2), B13-B22.
    doi: [10.1190/geo2014-0082.1](http://dx.doi.org/10.1190/geo2014-0082.1)

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = -30. # graus
dec = 0. # graus

In [4]:
amp = 23500.0 + 0.05*(xp - xc) + 0.08*(yp - yc) # nT

In [5]:
F = utils.ang2vec(1.0, inc, dec) # versor do campo geomagnético local

In [6]:
print F

[ 0.8660254  0.        -0.5      ]


### Modelo de prisma

In [7]:
mpl.close('all')
axis = mpl.figure().gca()
mpl.axis('scaled')
vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [8]:
print vertices

[[ 4027.19665272 -3661.08786611]
 [ 3870.29288703 -2379.70711297]
 [ 3661.08786611 -1046.0251046 ]
 [ 2850.41841004   758.36820084]
 [ 1385.9832636   2092.05020921]
 [ -156.90376569  2798.11715481]
 [-2275.10460251  3321.12970711]
 [-4001.0460251   3085.77405858]
 [-3399.58158996  2510.46025105]
 [-2248.9539749   2222.80334728]
 [  -52.30125523  1464.43514644]
 [ 1542.88702929  -496.86192469]
 [ 2458.15899582 -3425.73221757]
 [ 3530.33472803 -4550.20920502]
 [ 4079.49790795 -4367.15481172]]


In [9]:
incs = inc
decs = dec
m = 3.

model = [mesher.PolygonalPrism(vertices, 10, 1010, {'magnetization': utils.ang2vec(m, incs, decs)})]

### Amplitude do campo magnético anômalo

In [12]:
bx = polyprism.bx(xp, yp, zp, model)
by = polyprism.by(xp, yp, zp, model)
bz = polyprism.bz(xp, yp, zp, model)

In [13]:
amp_B = np.sqrt(bx**2 + by**2 + bz**2)

### Anomalia de campo total (em nT)

In [14]:
act_aprox = polyprism.tf(xp, yp, zp, model, inc, dec)

In [15]:
ct = np.sqrt((amp*F[0] + bx)**2 + (amp*F[1] + by)**2 + (amp*F[2] + bz)**2)

In [16]:
act = ct - amp

### Plot dos dados

In [17]:
mpl.close('all')
mpl.figure(figsize=(12,10))

mpl.subplot(2,2,1)
mpl.title("Campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, ct, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,2)
mpl.title("Anomalia de campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,3)
mpl.title("Anomalia de campo total aproximada (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act_aprox, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,4)
mpl.title("Amplitude do campo magnetico anomalo (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, amp_B, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout(pad = 2.)

mpl.savefig('prisma_mag_ampB.png', dpi=300)

#mpl.show()

<img src='prisma_mag_ampB.png', width=1200>